# 管理者の追加

---

CoursewareHubの管理ユーザをローカルユーザとして追加します。

## はじめに

このNotebookは、CoursewareHub環境構築の際に誤った管理者情報を登録したなどの理由により、CoursewareHub環境に管理者としてログインできるユーザが存在しないなどの状況を回避するためのものです。

## UnitGroup名

操作対象となるVCPのUnitGroup名を指定します。

VCノードを作成時に指定した値を確認するために `group_vars`ファイル名の一覧を表示します。

In [ ]:
!ls -1 --hide all group_vars/

操作対象となるUnitGroup名を次のセルに指定してください。

In [ ]:
# (例)
# ugroup_name = 'CoursewareHub'

ugroup_name = 

### チェック

対象となるVCノードがAnsibleによって操作できることを確認します。

Ansibleの設定ファイルの場所を環境変数に設定しておきます。

In [ ]:
from pathlib import Path
import os

cfg_ansible = Path('ansible.cfg')
if cfg_ansible.exists():
    os.environ['ANSIBLE_CONFIG'] = str(cfg_ansible.resolve())

構築対象となる各VCノードにアクセスできることを確認します。

In [ ]:
target_hub = f'{ugroup_name}_manager'

!ansible {target_hub} -m ping

In [ ]:
target_nodes = f'{ugroup_name}_worker'

!ansible {target_nodes} -m ping

UnitGroup名に対応する group_varsファイルが存在していることを確認します。

In [ ]:
!test -f group_vars/{ugroup_name}

## パラメータの設定

追加する管理者の情報を指定します。

管理者のメールアドレスを指定してください。

In [ ]:
# (例)
# teacher_email = 'admin@example.org'

teacher_email = 

管理者のパスワードを指定してください。

> ここで指定したパスワードは CoursewareHub にログインする際に必要となります。

In [ ]:
from getpass import getpass
teacher_password = getpass()

### パラメータの保存

管理者のメールアドレスを `group_vars`ファイルに保存します。

In [ ]:
import yaml
from pathlib import Path

gvars_path = Path(f'group_vars/{ugroup_name}')
with gvars_path.open() as f:
    gvars = yaml.safe_load(f)

teacher_email_list = gvars.get('teacher_email_list', [])
teacher_email_list.append(teacher_email)
teacher_email_list = list(set(teacher_email_list))

gvars.update({
    'teacher_email_list': teacher_email_list,
})

with gvars_path.open(mode='w') as f:
    yaml.safe_dump(gvars, stream=f)
    
!cat group_vars/{ugroup_name}

## 管理者の追加

CoursewareHubに管理ユーザを登録します。

ここでセットアップする主な項目を以下に示します。

* 全てのノード(manager, worker)にユーザーアカウントを作成する
* sshの鍵ペアを作成する
* ansibleのインベントリファイルを作成する
* sudo権限を与える
* JupyterHubユーザとして登録する

### パラメータの確認

CoursewareHubではメールアドレスから一定のルールで導き出された名前をローカルユーザ名として利用します。管理ユーザのローカルユーザ名を確認します。

In [ ]:
%run scripts/cwh.py
teacher_id = get_username_from_mail_address(teacher_email)
print(teacher_id)

管理ユーザのハッシュ化されたパスワードを取得します。

In [ ]:
from crypt import crypt

encrypted_teacher_password = crypt(teacher_password)
encrypted_teacher_password

JupyterHubのユーザ登録に必要となるcookie idを作成します。

In [ ]:
import random
import string
cookie_id = ''.join([random.choice("abcdef" + string.digits) for _ in range(32)])
cookie_id

### 管理ユーザの登録

ユーザに関するパラメータをextra_varsとして記述したJSONファイルを作成します。

In [ ]:
import tempfile
from pathlib import Path
import yaml

work_dir = tempfile.mkdtemp()
extra_vars = Path(work_dir) / 'group.yml'
with extra_vars.open(mode='w') as f:
    yaml.safe_dump(dict(
        teacher_id=teacher_id,
        cookie_id=cookie_id,
        encrypted_teacher_password=encrypted_teacher_password,
        teacher_email=teacher_email,
    ), stream=f)

managerノードに管理者ユーザを登録するplaybookを実行します。まずはチェックモードで確認を行います。

In [ ]:
!ansible-playbook -l {target_hub} -CDv -e @{str(extra_vars)} playbooks/setup-admin-user.yml || true

実際の設定変更を行います。

In [ ]:
!ansible-playbook -l {target_hub} -Dv -e @{str(extra_vars)} playbooks/setup-admin-user.yml

managerノードに作成した管理ユーザと同じアカウントをworkerノードにも作成します。managerノードに作成した管理ユーザのUID, GIDの値を確認します。

In [ ]:
out = !ansible {target_hub} -a 'id -u {teacher_id}' 2> /dev/null
teacher_uid = out[-1]
teacher_uid

In [ ]:
out = !ansible {target_hub} -a 'id -g {teacher_id}' 2> /dev/null
teacher_gid = out[-1]
teacher_gid

workerノードに管理者ユーザを登録するplaybookを実行します。まずはチェックモードで確認を行います。

In [ ]:
!ansible-playbook -l {target_nodes} -CDv \
    -e teacher_id={teacher_id} -e teacher_uid={teacher_uid} -e teacher_gid={teacher_gid} \
    playbooks/setup-worker-admin-user.yml

実際の設定変更を行います。

In [ ]:
!ansible-playbook -l {target_nodes} -Dv \
    -e teacher_id={teacher_id} -e teacher_uid={teacher_uid} -e teacher_gid={teacher_gid} \
    playbooks/setup-worker-admin-user.yml

### 確認

管理ユーザが作成されたことを確認します。

管理ユーザのホームディレクトリの状態を確認します。以下のファイルが作成されていることを確認してください。

|パス|内容|
|:---|:---|
|ansible/inventory|ansibleのインベントリ|
|.ssh/authorized_keys|公開鍵の登録ファイル|
|.ssh/id_rsa|sshの秘密鍵|
|.ssh/id_rsa.pub|sshの公開鍵|
|info/|コンテンツディレクトリ|
|textbook/|コンテンツディレクトリ|

In [ ]:
!ansible {target_hub} -b -a 'tree -aF {{{{home_dir}}}}/{teacher_id}'

JupyterHubのユーザとして登録されていることを確認します。

PostgreSQLコンテナでJupyterHubのユーザを管理している`users`テーブルと、CoursewareHubのローカル認証情報を管理している`local_users`の内容を表示します。それぞれのテーブルで`teacher_id`に対応するレコードが表示されることを確認してください。

In [ ]:
import json
out = !ansible-playbook -l {target_hub} -e teacher_id={teacher_id} \
    playbooks/show-jupyterhub-user.yml 2> /dev/null
line = [x for x in out if x.find("show_tables.stdout") >= 0][0]
print(json.loads(f'{{{line}}}')['show_tables.stdout'])

作業ディレクトリを削除します。

In [ ]:
!rm -rf {work_dir}

## コンテンツの配備の準備

CoursewareHubのコンテンツを格納するディレクトリを作成し、コンテンツの配置を行うNotebookを管理者のホームディレクトリに配置します。

ここで実行するplaybookが配置するファイル、ディレクトリを以下の表に示します。

|パス|用途|
|:---|:---|
|/jupyter/admin/admin_tools|コンテンツ用ディレクトリ|
|/jupyter/admin/tools|コンテンツ用ディレクトリ|
|/jupyter/admin/textbook|コンテンツ用ディレクトリ|
|/jupyter/admin/info|コンテンツ用ディレクトリ|
|/jupyter/users/{teacher_id}/D06_CoursewareHubコンテンツの配備.ipynb|コンテンツを配備するnotebook|

`{teacher_id}`は管理ユーザ名によって異なる値となります。

「D06_CoursewareHubコンテンツの配備.ipynb」はNII Cloud Operation Teamにより公開されている[CoursewareHub-LC_deploy/coursewarehub/D06_CoursewareHubコンテンツの配備.ipynb](https://github.com/NII-cloud-operation/CoursewareHub-LC_deploy/blob/main/coursewarehub/D06_CoursewareHub%E3%82%B3%E3%83%B3%E3%83%86%E3%83%B3%E3%83%84%E3%81%AE%E9%85%8D%E5%82%99.ipynb)をGitHubより取得して配備します。

実際に設定を変更する前にドライラン（チェックモード）でansibleを実行します。

In [ ]:
!ansible-playbook -l {target_hub} -CDv -e teacher_id={teacher_id} playbooks/manage-tools.yml

実際に設定変更を行います。

In [ ]:
!ansible-playbook -l {target_hub} -e teacher_id={teacher_id} playbooks/manage-tools.yml

配置したディレクトリ、ファイルを確認します。

In [ ]:
!ansible {target_hub} -b -a 'tree -F {{{{jupyter_root_dir}}}}/admin'

In [ ]:
!ansible {target_hub} -b -a 'tree -F -L 1 {{{{home_dir}}}}/{teacher_id}'

## CoursewareHubにアクセスする

追加した管理者ユーザでログインできることを確認してください。

次のセルを実行するとCoursewareHubのアドレスを表示します。

In [ ]:
print(f'https://{gvars["master_fqdn"]}')

管理ユーザのホームディレクトリにはコンテンツを配備するためのnotebook「D06_CoursewareHubコンテンツの配備.ipynb」が用意されています。必要に応じて実行してください。